In [ ]:
import requests

All data comes from https://cryptorank.io/. It has API but there is no endpoint for me to target. So the data has to be parsed from the page.

In [ ]:
headers = {
     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7,json", 
}

url = "https://api.cryptorank.io/v0/funds?limit=100&offset=0"